In [39]:
import arcpy
import pandas as pd
import numpy as np
from arcpy.sa import *

# 空间分配因子计算--栅格分配

## 1. 判断每个网格都属于哪个行政区。

此步骤通过相交分析来实现。

输入CMAQ网格（small_grid）和行政区网格(big_grid)来进行相交。

In [40]:
small_grid = 'E:\domain\Chengdu-3km\Domain_CMAQ\domain03_cmaq.shp'
big_grid = 'E:\全国矢量地图\国家基础地理信息系统\中国地州界.shp'
in_features = [small_grid, big_grid]
out_feature_class = f'F:\github\Haofan_Emission_Tools\output_directory\cmaq_intersect.shp'
arcpy.analysis.Intersect(in_features, out_feature_class, "ALL")

<Result 'F:\\github\\Haofan_Emission_Tools\\output_directory\\cmaq_intersect.shp'>

将输出文件属性表打开，仅保留ID,LAT,LON和行政区名称字段。

In [43]:
arcpy.CalculateGeometryAttributes_management(out_feature_class, [['square', 'AREA']], area_unit='SQUARE_KILOMETERS')

<Result 'F:\\github\\Haofan_Emission_Tools\\output_directory\\cmaq_intersect.shp'>

In [44]:
out_path = f'F:\github\Haofan_Emission_Tools\output_directory'
out_name = 'cmaq_intersect.csv'
arcpy.conversion.TableToTable(out_feature_class, out_path, out_name)

<Result 'F:\\github\\Haofan_Emission_Tools\\output_directory\\cmaq_intersect.csv'>

此代码将所有小网格都唯一对应在了一个大网格的名称下。

In [45]:
df = pd.read_csv(f'{out_path}/{out_name}')
df_ids = df['ID'].values
df_area = df['square'].values
del_list = []
for temp_id in range(1, df_ids.max()+1):
    pos_in_df = np.where(df_ids == temp_id)[0]
    temp_area = df_area[pos_in_df]
    if len(temp_area) > 1.0:
        pos_in_temp = np.where(temp_area != temp_area.max())
        pos = pos_in_df[pos_in_temp]
        for temp_pos in pos:
            del_list.append(temp_pos)
df = df.drop(del_list)
df

,OID_,ID,LON,LAT,NAME,square
0,0,1,101.576029,28.399021,康定县,9.000000
1,1,2,101.606922,28.399710,康定县,9.000000
2,2,3,101.637815,28.400390,康定县,9.000000
3,3,4,101.668708,28.401061,康定县,9.000000
4,4,5,101.699602,28.401724,康定县,9.000000
...,...,...,...,...,...,...
26776,26776,24645,106.391070,32.680985,汉中市,9.000000
26777,26777,24646,106.423582,32.680311,汉中市,9.000000
26778,26778,24647,106.456093,32.679627,汉中市,9.000000
26779,26779,24648,106.488604,32.678935,汉中市,7.737879


## 进行栅格分配

### 计算每个行政区的栅格总值

In [46]:
# Clip
clipped_big_grid = f'F:\github\Haofan_Emission_Tools\output_directory\Big_grid_cliped.shp'
arcpy.Clip_analysis(big_grid, small_grid, clipped_big_grid)

<Result 'F:\\github\\Haofan_Emission_Tools\\output_directory\\Big_grid_cliped.shp'>

In [47]:
raster_file = r'E:\Model\build_emission\建立全国排放因子数据库\全国土地利用类型数据\2015\CLUDA_2015_10_1km_prop.tif'
# statistic
out_statistic_table = f'F:\github\Haofan_Emission_Tools\output_directory\Big_grid_zonalstattblout.dbf'
outZSaT = ZonalStatisticsAsTable(clipped_big_grid, "NAME", raster_file, out_statistic_table, statistics_type="SUM")
out_path = f'F:\github\Haofan_Emission_Tools\output_directory'
out_name = 'Big_grid_zonalstattblout.csv'
arcpy.conversion.TableToTable(out_statistic_table, out_path, out_name)

<Result 'F:\\github\\Haofan_Emission_Tools\\output_directory\\Big_grid_zonalstattblout.csv'>

### 计算每个小网格中的栅格总数

In [48]:
# statistic
out_statistic_table = f'F:\github\Haofan_Emission_Tools\output_directory\Small_grid_zonalstattblout.dbf'
outZSaT = ZonalStatisticsAsTable(small_grid, "ID", raster_file, out_statistic_table, statistics_type="SUM")
out_path = f'F:\github\Haofan_Emission_Tools\output_directory'
out_name = 'Small_grid_zonalstattblout.csv'
arcpy.conversion.TableToTable(out_statistic_table, out_path, out_name)

<Result 'F:\\github\\Haofan_Emission_Tools\\output_directory\\Small_grid_zonalstattblout.csv'>

### 以完整的网格信息表(df)来作为基准进行排放因子统计

In [51]:
big_grid_zstt = pd.read_csv('F:\github\Haofan_Emission_Tools\output_directory\Big_grid_zonalstattblout.csv')
big_grid_zstt

,OID_,NAME,ZONE_CODE,COUNT,AREA,SUM
0,0,西昌市,1.0,11817.0,0.820625,1892.600592
1,1,昭通市,2.0,1656.0,0.115000,535.060651
2,2,宜宾市,3.0,11486.0,0.797639,7819.405325
3,3,乐山市,4.0,17324.0,1.203056,7637.332840
4,4,自贡市,5.0,5887.0,0.408819,4421.498521
5,5,内江市,6.0,7296.0,0.506667,6226.136095
6,6,遵义市,7.0,2406.0,0.167083,613.752959
7,7,遂宁县,8.0,7219.0,0.501319,5832.155325
8,8,华蓥市,9.0,2545.0,0.176736,2240.155325
9,9,泸州市,10.0,8321.0,0.577847,5892.270710


In [52]:
small_grid_zstt = pd.read_csv('F:\github\Haofan_Emission_Tools\output_directory\Small_grid_zonalstattblout.csv')
small_grid_zstt

,OID_,ID,ZONE_CODE,COUNT,AREA,SUM
0,0,1,1,9.0,0.000625,0.974852
1,1,2,2,12.0,0.000833,2.927515
2,2,3,3,12.0,0.000833,1.900888
3,3,4,4,12.0,0.000833,4.096154
4,4,5,5,10.0,0.000694,3.034024
...,...,...,...,...,...,...
24644,24644,24645,24645,12.0,0.000833,5.696746
24645,24645,24646,24646,12.0,0.000833,5.375740
24646,24646,24647,24647,12.0,0.000833,3.173077
24647,24647,24648,24648,16.0,0.001111,1.547337


In [70]:
ef_list = []
id_list = []
area_list = []
lon_list = []
lat_list = []
for temp_df in df.values:
    temp_id = temp_df[1]
    temp_area = temp_df[4]
    temp_sum = temp_df[5]
    temp_lon = temp_df[2]
    temp_lat = temp_df[3]
#     print(temp_id, temp_area, temp_sum)
    temp_small_grid_length = small_grid_zstt.values[temp_id-1][5]
    temp_big_grid_length = big_grid_zstt[big_grid_zstt['NAME'].isin([temp_area])]['SUM'].values[0]
    temp_ef = temp_small_grid_length/temp_big_grid_length
    ef_list.append(temp_ef)
    id_list.append(temp_id)
    area_list.append(temp_area)
    lon_list.append(temp_lon)
    lat_list.append(temp_lat)

result = pd.DataFrame(columns=['ID', 'LON', 'LAT', 'AREA', 'EF'])
result['ID'] = id_list
result['LON'] = lon_list
result['LAT'] = lat_list
result['AREA'] = area_list
result['EF'] = ef_list

result

,ID,LON,LAT,AREA,EF
0,1,101.576029,28.399021,康定县,0.001319
1,2,101.606922,28.399710,康定县,0.003962
2,3,101.637815,28.400390,康定县,0.002572
3,4,101.668708,28.401061,康定县,0.005543
4,5,101.699602,28.401724,康定县,0.004106
...,...,...,...,...,...
24644,24645,106.391070,32.680985,汉中市,0.120302
24645,24646,106.423582,32.680311,汉中市,0.113523
24646,24647,106.456093,32.679627,汉中市,0.067008
24647,24648,106.488604,32.678935,汉中市,0.032676


In [72]:
result_csv_path = 'F:\github\Haofan_Emission_Tools\output_directory\EF_AG.csv'
result.to_csv(result_csv_path, index=False)

In [73]:
result_point_path = 'F:\github\Haofan_Emission_Tools\output_directory\EF_AG.shp'
arcpy.management.XYTableToPoint(result_csv_path, result_point_path, 'LON', 'LAT', z_field='EF')

<Result 'F:\\github\\Haofan_Emission_Tools\\output_directory\\EF_AG.shp'>